In [1]:
import sys
sys.path.append("./../src")
sys.path.append("./../models")

from api import DataApi
# from config import DATA
from helpers import DataCleansing, weighted_rating
from model import ContentBasedF


import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
API = DataApi("movies")
response = API.READ_DATA()

In [7]:
USE_COLS = ['movieId', 'genres', 'meta.rate_counts', 'meta.rate_average']
test = pd.json_normalize(response)[USE_COLS].dropna(subset=['meta.rate_counts'])

models = ContentBasedF(test, 4)
models.fit()
models.recommend()

Empty DataFrame
Columns: [movieId, genres, meta.rate_counts, meta.rate_average]
Index: []
Data Preprocess Done 
Calculate Similarity Done
Data Preprocess Done 
Calculate Similarity Done


array([  6370,  53318,    803, 133767,  55844, 142929, 185347,  90359,
        96757, 146140])

In [6]:
models._matrix.

<58303x24 sparse matrix of type '<class 'numpy.int64'>'
	with 118203 stored elements in Compressed Sparse Row format>

In [ ]:
matrix = pd.read_pickle("./item_user_matrix.pkl")
matrix.head()

userId,2,3,4,5,8,9,12,13,18,19,...,162519,162521,162524,162529,162532,162533,162534,162537,162538,162541
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# api = DataApi("movies")
# MOVIE_DB = pd.DataFrame(api.READ_DATA())
# MOVIE_DB.movieId = MOVIE_DB.movieId.astype(int)
MOVIE_DB = pd.read_csv(DATA + 'movies.csv')
#MOVIE_DB = MOVIE_DB[MOVIE_DB['year'] >= 1970]
RATE_DB = pd.read_csv(DATA + "ratings.csv", usecols=['userId', 'movieId', 'rating'])

In [8]:
!pip install Flask --quiet
!pip install flask-ngrok --quiet

In [9]:
from models.recommender import ContentBasedF
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok
from joblib import load

query_id = 1544

def requestResults(query_id):
    model = ContentBasedF(MOVIE_DB, RATE_DB, query_id)
    return model.recommend()

# start Flask
app = Flask(__name__)
run_with_ngrok(app)

# render default webpage
@app.route('/')
def home():
    return render_template('home.html')

# post method return
@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        id = request.form['search']
        return redirect(url_for('success', query_id=id))

# get the data for requested query
@app.route('/success/<query_id>')
def success(query_id):
    return requestResults(query_id)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0719-34-27-220-75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-9-820e708ba77e>", line 19, in home
    return render_template('home.html')
  File "/usr/local/lib/python3.10/dist-packages/flask/templating.py", line 146, in render_template
    template = app.jinja_env.get_or_select_template(template_name_or_list)
  File "/usr/local/lib/python3.10/dist-packages/jinja2/environment.py", line 1081, in 

In [ ]:
# 컨텐츠 기반 필터링
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Garbage Collector - use it like gc.collect()
import gc

class ContentBasedF:
    def __init__(self, data, rates, query_id):
        self.data = data
        self.rates = rates
        self.popularity = None
        self._query_id = query_id
        self._matrix = None
        self._sim = None

    @property
    def matrix(self):
        return self._matrix

    @property
    def sim(self):
        return self._sim

    def _calculate_popularity(self):
        rate_counts = self.rates.groupby("movieId").count()['rating'].reset_index()
        rate_average = self.rates.groupby("movieId").mean()['rating'].reset_index()

        popularity = (pd.merge(rate_counts, rate_average, 'inner', 'movieId')
                        .rename({'rating_x' : 'rate_counts', 'rating_y' : 'rate_average'}, axis=1))

        gc.collect()
        return popularity


    def _calculate_weighted_votes(self, f=1):
        popularity = self._calculate_popularity()
        weighted_rates = weighted_rating(popularity['rate_counts'], popularity['rate_average'], f=1)
        popularity['weighted_rating'] = weighted_rates

        print("Calculate Weighted Votes Done ")

        return popularity

    def preprocess(self, weighted_vote=False, f=1):
        self.data['new_genre'] = self.data['genres'].str.replace('|', ' ', regex=False)

        if weighted_vote:
             self.popularity = self._calculate_weighted_votes(f)
             self.data = pd.merge(self.data, self.popularity, how = 'inner', on='movieId')

        gc.collect()
        print("Data Preprocess Done ")

    def vectorize(self):
        self._matrix = self._vectorizer.fit_transform(self.data['new_genre'])
        print("Model Fit Done ")

    def fit(self):
        # calculate weighted vote and split genres
        self.preprocess(weighted_vote=True, f=1)

        # make countvector to genre
        vectorizer = CountVectorizer(dtype=np.int16)
        self._matrix = vectorizer.fit_transform(self.data['new_genre'])

        query_genre = self.data[self.data['movieId'] == self._query_id]['new_genre']
        query_vec = vectorizer.transform(query_genre)

        # calculate cosine similarity between one movie and cosine matrix
        self._sim = cosine_similarity(query_vec, self._matrix).flatten()
        print("Calculate Similarity Done")
        gc.collect()

    def recommend(self, top_n=10):
        self.fit()
        sorted_ind = self._sim.argsort()[-(top_n * 2):][::-1]
        title_index = self.data.loc[self.data['movieId'] == self._query_id].index.values

        similar_indexes = sorted_ind[sorted_ind != title_index]
        return self.data.iloc[similar_indexes].sort_values('weighted_rating', ascending=False)[:top_n].movieId.values

In [ ]:
cbf = ContentBasedF(MOVIE_DB, RATE_DB, 1544)
# cbf.fit(1544)
cbf.recommend()

Calculate Weighted Votes Done 
Data Preprocess Done 
Calculate Similarity Done


array([122882,    480,   2916,   8644,    780, 195423, 172891,  68791,
        61248, 191169])

# COLLABORATIVE FILTERING

## Item-Based

In [ ]:
from scipy.spatial.distance import cosine, pdist, squareform

class ItemBasedCF:
    def __init__(self, movie, rates, user_item_matrix=None):
        self._movies = movie
        self._rates = rates
        self._user_item_matrix = user_item_matrix

    def _calculate_popularity(self):
        rate_counts = self._rates.groupby("movieId").count()['rating'].reset_index()
        rate_average = self._rates.groupby("movieId").mean()['rating'].reset_index()

        popularity = (pd.merge(rate_counts, rate_average, 'inner', 'movieId')
                        .rename({'rating_x' : 'rate_counts', 'rating_y' : 'rate_average'}, axis=1))


        gc.collect()
        return popularity

    def _filtering_movies(self):
        popularity = self._calculate_popularity()

        count_idx_ = popularity.loc[popularity['rate_counts'] > 1000, 'movieId'].values
        top_movie = self._movies.loc[self._movies['movieId'].isin(count_idx_)]

        top_users = self._rates.groupby('userId').count()['movieId'].reset_index().sort_values('movieId', ascending=False).rename({'movieId' : 'user_counts'}, axis=1)
        user_idx_ = top_users.loc[top_users['user_counts'] > 50, 'userId'].values

        gc.collect()
        return top_movie, user_idx_

    def make_user_item_matrix(self, save=False):
        # filtering users and movies
        movie, user = self._filtering_movies()

        table = (
            pd.merge(movie, self._rates.loc[self._rates['userId'].isin(user)], 'inner', 'movieId')
            .drop_duplicates()
            .reset_index()
        )

        user_item_matrix = table.groupby(['userId', 'title'])['rating'].max().unstack()
        user_item_matrix.fillna(0, inplace=True)

        if save:
            user_item_matrix.to_pickle("./user_item_matrix.pkl")
            print("Save Done")

        print(f"user_item_matrix fit Done, shape: {user_item_matrix.shape}")
        gc.collect()
        return user_item_matrix

    # user_item matrix 기준
    def fit(self):

        # user_item matrix 기준
        M = self._user_item_matrix.values
        print(M.shape)
        M_u = M.mean(axis=1)
        print(M_u, M_u.shape)
        item_mean_subtracted = M - M_u[:,None] # shape을 2d로 해줌
        print(item_mean_subtracted.shape)
        gc.collect()

        # to reduce ram usage, change dtype float32 to float16
        similarity_matrix = 1 - squareform(pdist(item_mean_subtracted.T.astype(np.float16), 'cosine'))
        gc.collect()
        return similarity_matrix

user_item_matrix = pd.read_pickle("./user_item_matrix.pkl")
ibf = ItemBasedCF(MOVIE_DB, RATE_DB, user_item_matrix)
ibf.fit()

In [ ]:
class UserBasedCF:
    def __init__(self, ):
        self.

In [ ]:
from scipy.spatial.distance import cosine, pdist, squareform, cdist

# title - user matrix 기준
def calculate_adjust_cosine_similarity(item_user_matrix):
    M = np.asarray(item_user_matrix.loc[item_user_matrix.index, :])
    M_u = M.mean(axis=1)
    item_mean_subtracted = M - M_u[:,None] # shape을 2d로 해줌
    print(item_mean_subtracted.shape)

    # to reduce ram usage, change dtype float32 to float16
    similarity_matrix = 1 - squareform(pdist(item_mean_subtracted.astype(np.float16), 'cosine'))
    return similarity_matrix

In [ ]:
test_matrix = matrix.iloc[:, :30]
test_matrix.head()

userId,2,3,4,5,8,9,12,13,18,19,...,59,61,62,67,69,70,72,75,76,80
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_item_matrix = test_matrix.T
user_user_similarity = calculate_adjust_cosine_similarity(user_item_matrix)

(30, 3786)


In [ ]:
test_matrix.shape[0]

3786

In [ ]:
new_user_vector = np.zeros((test_matrix.shape[0], 1))
new_user_vector[[10, 88, 90, 624, 1003, 1038]] = 3.5
new_user_vector[[5, 99, 746]] = 4.0
M = np.asarray(new_user_vector)
M_u = np.mean(M)
new_user_item_mean_subtracted = M - M_u # shape을 2d로 해줌
print(new_user_item_mean_subtracted.shape)

(3786, 1)


In [ ]:
item_user_matrix = matrix
M = np.asarray(item_user_matrix.loc[item_user_matrix.index, :])
M_u = M.mean(axis=1)
item_mean_subtracted = M - M_u[:,None] # shape을 2d로 해줌
print(item_mean_subtracted.shape)

(3786, 63102)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(item_mean_subtracted.astype(np.float16).T, new_user_item_mean_subtracted)
similarity

ValueError: ignored

In [ ]:
similarity_matrix = 1 - squareform(cdist(item_mean_subtracted.astype(np.float16), new_user_item_mean_subtracted,'cosine'))
similarity_matrix.shape

ValueError: ignored

In [ ]:
def similarity(item_user_matrix, new_user_vector):
    M = np.asarray(new_user_vector)
    M_u = np.mean(M)
    item_mean_subtracted = M - M_u[:,None] # shape을 2d로 해줌
    print(item_mean_subtracted.shape)

    # to reduce ram usage, change dtype float32 to float16
    # similarity_matrix = 1 - squareform(pdist(item_mean_subtracted.astype(np.float16), 'cosine'))
    # return similarity_matrix

KeyError: ignored

In [ ]:
user_user = pd.DataFrame(user_user_similarity, index = test_matrix.columns, columns = test_matrix.columns)
user_id = 2
query_user_id = 19
round(user_user.loc[user_id, query_user_id], 3)

0.149

In [ ]:
matrix = pd.read_pickle("./item_user_matrix.pkl")
matrix.head()

userId,2,3,4,5,8,9,12,13,18,19,...,162519,162521,162524,162529,162532,162533,162534,162537,162538,162541
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.spatial.distance import cosine, pdist, squareform

# title - user matrix 기준
def calculate_adjust_cosine_similarity(item_user_matrix):
    M = np.asarray(item_user_matrix.loc[item_user_matrix.index, :])
    M_u = M.mean(axis=1)
    item_mean_subtracted = M - M_u[:,None] # shape을 2d로 해줌
    print(item_mean_subtracted.shape)

    # to reduce ram usage, change dtype float32 to float16
    similarity_matrix = 1 - squareform(pdist(item_mean_subtracted.astype(np.float16), 'cosine'))
    return similarity_matrix

In [ ]:
item_sim_df = pd.DataFrame(similarity_matrix, index=matrix.index, columns=matrix.index)
item_sim_df.shape

(3786, 3786)

In [ ]:
item_sim_df[item_sim_df.columns.str.contains("Dark Knight")].iloc[0].sort_values(ascending=False)[:10]

title
Dark Knight Rises, The (2012)     1.000000
Avengers, The (2012)              0.647054
Dark Knight, The (2008)           0.639040
Inception (2010)                  0.630989
Django Unchained (2012)           0.607151
Interstellar (2014)               0.599187
Guardians of the Galaxy (2014)    0.590583
Edge of Tomorrow (2014)           0.540995
Mad Max: Fury Road (2015)         0.534366
X-Men: First Class (2011)         0.532218
Name: Dark Knight Rises, The (2012), dtype: float64

In [ ]:
ratings_pred = matrix.values.T.dot(similarity_matrix) / np.array([np.abs(similarity_matrix).sum(axis=1)])
ratings_pred_matrix = pd.DataFrame(ratings_pred, index=matrix.columns, columns=matrix.index)

In [ ]:
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(3)

(63102, 3786)


title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),...,Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
2,0.193179,0.098455,0.212036,0.156055,0.087832,0.224100,0.167751,0.216993,0.238796,0.172462,...,0.145753,0.118185,0.237713,0.078238,0.224417,0.101463,0.171336,0.225434,0.166292,0.233506
3,0.382104,1.186180,0.609924,0.275206,1.270404,0.679855,1.216651,0.506899,0.455616,0.681990,...,1.190337,1.346797,1.017034,1.254415,0.442598,1.037665,0.612442,1.054774,1.101361,0.334301
4,0.120627,0.440652,0.180091,0.066350,0.542190,0.187901,0.355769,0.137270,0.153265,0.170572,...,0.419380,0.541119,0.349039,0.577680,0.142080,0.355698,0.166305,0.287008,0.294090,0.113025


In [ ]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print(get_mse(ratings_pred, matrix.values.T))

9.477982783014134


In [ ]:
#item_sim_df.to_pickle("./adjust_similarity_item.pkl")


In [ ]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    pred = np.zeros(ratings_arr.shape)

    for col in range(ratings_arr.shape[1]):
        top_n_items = [np.argsort(item_sim_arr[:,col])[:-n-1:-1]]
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred

ratings_pred = predict_rating_topsim(matrix.values.T, similarity_matrix)
print(get_mse(ratings_pred, matrix.values.T))

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=matrix.column, columns = matrix.index)

KeyboardInterrupt: ignored